In [37]:
import numpy as np
import pandas as pd
import re
import tensorflow
from keras.models import Sequential
from keras import layers
from tensorflow.keras.preprocessing.text import one_hot,Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [38]:
data = pd.read_csv('data.txt')

In [39]:
data.head()

,1
0,ای رستخیز ناگهان، وی رحمت بی منتها\tای آتشی اف...
1,امروز خندان آمدی، مفتاح زندان آمدی\tبر مستمندا...
2,خورشید را حاجب تویی، امید را واجب تویی\tمطلب ت...
3,در سینه ها برخاسته، اندیشه را آراسته\tهم خویش ...
4,ای روح بخش بی بَدَل، وی لذتِ علم و عمل\tباقی ب...


In [40]:
with open('stopwords.txt') as stopwords_file:
  stopwords = stopwords_file.readlines()
stopwords = [line.replace('\n', '') for line in stopwords] 

In [41]:
stopwords[0:20]

['!',
 '"',
 '#',
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 ':',
 '[',
 ']',
 '«',
 '»',
 '،',
 '؛',
 '؟',
 'آباد',
 'آخ']

In [42]:
len(stopwords)

1316

In [45]:
text = [line.replace('\t', ' ') for line in data.values.flatten()]
text = [line.replace('-', ' ') for line in text]
text = [re.sub("\d+", "", t) for t in text]

In [46]:
text

['ای رستخیز ناگهان، وی رحمت بی منتها ای آتشی افروخته، در بیشه ی اندیشه ها',
 'امروز خندان آمدی، مفتاح زندان آمدی بر مستمندان آمدی، چون بخشش و فضل خدا',
 'خورشید را حاجب تویی، امید را واجب تویی مطلب تویی طالب تویی، هم منتها، هم مبتدا',
 'در سینه ها برخاسته، اندیشه را آراسته هم خویش حاجت خواسته، هم خویشتن کرده روا',
 'ای روح بخش بی بَدَل، وی لذتِ علم و عمل باقی بهانه ست و دغل، کاین علت آمد، وآن دوا',
 'ما زان دغل کژ بین شده، با بی گنه در کین شده گه مست حورالعین شده، گه مست نان و شوربا',
 'این سُکر بین، هل عقل را، وین ُنقل بین، هل َنقل را کز بهر نان و بقل را، چندین نشاید ماجرا',
 'تدبیر صد رنگ افکنی، بر روم و بر زنگ افکنی واندر میان جنگ افکنی، فی اصطناع لا یری',
 'میمال پنهان گوش جان، مینه بهانه بر کسان جان رب خلصنی زنان، والله که لاغست ای کیا',
 'خامش که بس مستعجلم، رفتم سوی پای علم کاغذ بنه بشکن قلم، ساقی درآمد، الصلا',
 '',
 'ای طایران قدس را عشقت فزوده بال ها در حلقه ی سودای تو، روحانیان را حال ها',
 'در "لا احب الافلین"، پاکی ز صورت ها یقین در دیده های غیب بین، هر دم ز تو تمثال ها',


In [47]:
!pip install hazm
from hazm import word_tokenize

In [48]:
word_tokenized = [word_tokenize(t) for t in text]
word_tokenized_filtered = [[w for w in sentence if w not in stopwords] for sentence in word_tokenized]

In [49]:
#example
word_tokenized_filtered[14]

['کوه',
 'غمت',
 'بشکافته',
 'وآن',
 'غم',
 'دل',
 'درتافته',
 'قطره',
 'خونی',
 'فضلت',
 'افضال']

In [50]:
vocab_len = 0
sentences = []
for sentence in word_tokenized_filtered:
    vocab_len += len(sentence)
    sentences.append(' '.join(sentence))

In [51]:
vocab_len

42271

In [52]:
sentences = [sentence for sentence in sentences if sentence != '']

In [53]:
len(sentences)

5316

In [54]:
sentences[0]

'رستخیز رحمت منتها آتشی افروخته بیشه اندیشه'

In [55]:
encoded_vocab = [one_hot(sentence, vocab_len) for sentence in sentences] 

In [56]:
len(encoded_vocab)

5316

In [57]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

In [58]:
import pickle
with open('tokenizer.h5', 'wb') as f:
    pickle.dump(tokenizer, f)

In [59]:
tokenizer.word_index

{'جان': 1,
 'دل': 2,
 'عشق': 3,
 'ست': 4,
 'گر': 5,
 'آب': 6,
 'چشم': 7,
 'اندر': 8,
 'شب': 9,
 'جهان': 10,
 'شمس': 11,
 'دست': 12,
 'مست': 13,
 'گل': 14,
 'کز': 15,
 'یار': 16,
 'زان': 17,
 'عقل': 18,
 'جمله': 19,
 'کان': 20,
 'غم': 21,
 'آتش': 22,
 'نور': 23,
 'مه': 24,
 'نی': 25,
 'پر': 26,
 'خاک': 27,
 'تن': 28,
 'عالم': 29,
 'آنک': 30,
 'تبریز': 31,
 'کار': 32,
 'خدا': 33,
 'شاه': 34,
 'ره': 35,
 'خانه': 36,
 'باده': 37,
 'همچو': 38,
 'عاشق': 39,
 'تویی': 40,
 'زین': 41,
 'روح': 42,
 'فی': 43,
 'ماه': 44,
 'ساقی': 45,
 'خون': 46,
 'همی': 47,
 'کف': 48,
 'رخ': 49,
 'لا': 50,
 'لطف': 51,
 'شکر': 52,
 'مر': 53,
 'پا': 54,
 'زر': 55,
 'وز': 56,
 'ای': 57,
 'آسمان': 58,
 'گوش': 59,
 'وان': 60,
 'عاشقان': 61,
 'بهر': 62,
 'زمین': 63,
 'بحر': 64,
 'ذره': 65,
 'خورشید': 66,
 'کیست': 67,
 'شه': 68,
 'سخن': 69,
 'چرخ': 70,
 'کاین': 71,
 'گرد': 72,
 'پرده': 73,
 'باغ': 74,
 'بادا': 75,
 'ور': 76,
 'جام': 77,
 'تبریزی': 78,
 'دین': 79,
 'چونک': 80,
 'آرزوست': 81,
 'پای': 82,
 'خواب': 83,
 'آف

In [60]:
encoded = tokenizer.texts_to_sequences(sentences)

In [61]:
encoded

[[2792, 328, 1400, 450, 1676, 1029, 177],
 [220, 629, 1185, 451, 629, 4350, 629, 892, 379, 33],
 [66, 2793, 40, 699, 2097, 40, 893, 40, 302, 40, 1400, 4351],
 [122, 4352, 177, 1401, 787, 788],
 [42, 4353, 4354, 263, 196, 452, 4, 1677, 71, 1030, 2794, 287],
 [17, 1677, 789, 2098, 630, 13, 4355, 13, 251, 2795],
 [4356, 521, 18, 221, 4357, 521, 4358, 15, 62, 251, 4359, 1186, 473],
 [1031, 134, 1678, 400, 1187, 1678, 4360, 222, 1678, 43, 4361, 50, 790],
 [4362, 107, 59, 1, 4363, 452, 1679, 1, 303, 4364, 288, 329, 4365, 1188],
 [241, 4366, 791, 82, 263, 2796, 428, 792, 1189, 45, 631, 223],
 [4367, 1680, 522, 2797, 523, 178, 330, 4368],
 [50, 2099, 2798, 1681, 264, 184, 2799],
 [700, 2100, 27, 265, 46, 1402, 4369, 1403, 44, 632],
 [179, 2800, 2801, 2794, 21, 2, 4370, 353, 1190, 4371, 4372],
 [4373, 4374, 4375, 17, 1191, 242, 4376, 8, 2802, 4377],
 [567, 2803, 1404, 4378, 429, 1, 4379, 4380, 85, 453],
 [523, 1682, 2804, 2101],
 [2102, 118, 4381, 118, 4382, 4383, 4384, 55, 4385, 135, 2103],
 [

In [62]:
padded = pad_sequences(encoded, maxlen=50, padding='post')
padded
padded.shape

(5316, 50)

In [63]:
# max_length = 15
# padded_reviews = pad_sequences(encoded_vocab, maxlen=max_length, padding='post')
# print(padded_reviews)

In [64]:
X_train = padded
y_train = padded

In [65]:
max_len = 1000
model = Sequential()
model.add(layers.Embedding(vocab_len, 50,input_shape=(vocab_len,)))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(vocab_len, activation='softmax'))

In [66]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 42271, 50)         2113550   
                                                                 
 flatten_2 (Flatten)         (None, 2113550)           0         
                                                                 
 dense_4 (Dense)             (None, 10)                21135510  
                                                                 
 dense_5 (Dense)             (None, 42271)             464981    
                                                                 
Total params: 23,714,041
Trainable params: 23,714,041
Non-trainable params: 0
_________________________________________________________________


In [67]:
model.fit([padded], np.array(y_train), epochs=10, verbose=0, batch_size=16,validation_split=0.2)

ValueError: ignored